# Menjalankan QA Tanpa Intermediate Task - Transfer Learning

# Import semua module

In [1]:
#!pip install datasets
#!pip install transformers
#!pip install tensorboard
#!pip install evaluate
#!pip install git+https://github.com/IndoNLP/nusa-crowd.git@release_exp

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [3]:
!pip install --upgrade pip
!pip install -r requirements.txt

  Cloning https://github.com/IndoNLP/nusa-crowd.git (to revision 7748513d20331e72f9969f94f5d43c7f2d4a59a5) to /tmp/pip-install-_3umgsaz/nusacrowd_5508d0472177444fbb9066491fe46d18
  Running command git clone --filter=blob:none --quiet https://github.com/IndoNLP/nusa-crowd.git /tmp/pip-install-_3umgsaz/nusacrowd_5508d0472177444fbb9066491fe46d18
  Running command git rev-parse -q --verify 'sha^7748513d20331e72f9969f94f5d43c7f2d4a59a5'
  Running command git fetch -q https://github.com/IndoNLP/nusa-crowd.git 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Running command git checkout -q 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Resolved https://github.com/IndoNLP/nusa-crowd.git to commit 7748513d20331e72f9969f94f5d43c7f2d4a59a5
  Preparing metadata (setup.py) ... done
  Using cached torch-1.13.1-cp38-cp38-manylinux1_x86_64.whl (887.4 MB)
  Using cached torchaudio-0.13.1-cp38-cp38-manylinux1_x86_64.whl (4.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninst

In [4]:
# Melihat GPU yang tersedia dan penggunaannya.
!nvidia-smi

Sun Mar 26 10:06:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   36C    P0    57W / 300W |  32476MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   60C    P0   295W / 300W |  32292MiB / 32480MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [5]:
# Memilih GPU yang akan digunakan (contohnya: GPU #7)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [6]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys
import contextlib

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    EvalPrediction,
    AutoModel,
    BertModel
)

# Definisikan hyperparameter

In [7]:
#MODEL_NAME = "indolem/indobert-base-uncased"
#MODEL_NAME = "afaji/fine-tuned-IndoNLI-Translated-with-indobert-base-uncased"
MODEL_NAME = "afaji/fine-tuned-DatasetQAS-TYDI-QA-ID-with-indobert-base-uncased-with-ITTL-without-freeze-LR-1e-05"
#MODEL_NAME = "indobenchmark/indobert-large-p2"
SEED = 42
EPOCH = 1
BATCH_SIZE = 16
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 1e-5
MAX_LENGTH = 400
STRIDE = 100
LOGGING_STEPS = 50
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01
MAXIMUM_SEARCH_ITER =  2
# Untuk mempercepat training, saya ubah SAMPLE menjadi 100.
# Bila mau menggunakan keseluruhan data, gunakan: 
SAMPLE = sys.maxsize
# SAMPLE = 50

# Import dataset QAS

In [8]:
#!pip install git+https://github.com/IndoNLP/nusa-crowd.git

In [9]:
#!pip install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html

In [10]:
#!pip uninstall 

In [11]:
import torch

print(torch.version.cuda)

11.7


In [12]:
#!pip install tensorflow

In [13]:
#import tensorflow as tf
#from tensorflow.python.platform import build_info as build
#print(f"tensorflow version: {tf.__version__}")
#print(f"Cuda Version: {build.build_info['cuda_version']}")
#print(f"Cudnn version: {build.build_info['cudnn_version']}")

In [14]:
#!pip list

In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_Sep_13_19:13:29_PDT_2021
Cuda compilation tools, release 11.5, V11.5.50
Build cuda_11.5.r11.5/compiler.30411180_0


In [16]:
#!pip uninstall torch
#!pip install torch

In [17]:
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2

In [18]:
!nvidia-smi

Sun Mar 26 10:06:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   36C    P0    57W / 300W |  32476MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   59C    P0   242W / 300W |  32248MiB / 32480MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [19]:
#!pip uninstall -y torch

In [20]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [21]:
!pip install cudatoolkit==11.5

ERROR: Could not find a version that satisfies the requirement cudatoolkit==11.5 (from versions: none)
ERROR: No matching distribution found for cudatoolkit==11.5


In [22]:
!pip install conda

In [23]:
!conda install cudatoolkit=11.5

ERROR: The install method you used for conda--probably either `pip install conda`
or `easy_install conda`--is not compatible with using conda as an application.
If your intention is to install conda as a standalone application, currently
supported install methods include the Anaconda installer and the miniconda
installer.  You can download the miniconda installer from
https://conda.io/miniconda.html.



In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:88: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [25]:
torch._C._cuda_getDeviceCount()

0

In [26]:
torch.cuda.is_available()

False

In [27]:
device

device(type='cpu')

In [28]:
print(torch.__version__)

1.13.1+cu117


# Definisikan tokenizer

In [29]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Definisikan fungsi pre-processnya

In [30]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas_id['train'])
df_validation = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    for j in df_validation["qas"][i]:
        if len(j['answers']) != 0:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": j['answers'][0]['text'], 
                                                       "answer_start": j['answers'][0]['answer_start'], 
                                                       "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                       ignore_index=True)
        else:
            new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                            'question': j['question'], 
                                            'answer': {"text": str(), 
                                                       "answer_start": 0, 
                                                       "answer_end": 0}}, 
                                                       ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    for j in df_train["qas"][i]:
        if len(j['answers']) != 0:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                                'question': j['question'], 
                                                'answer': {"text": j['answers'][0]['text'], 
                                                           "answer_start": j['answers'][0]['answer_start'], 
                                                           "answer_end": j['answers'][0]['answer_start'] + len(j['answers'][0]['text'])}}, 
                                                           ignore_index=True)
        else:
            new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                                'question': j['question'], 
                                                'answer': {"text": str(), 
                                                           "answer_start": 0, 
                                                           "answer_end": 0}}, 
                                                           ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)
validation_dataset = Dataset.from_dict(new_df_val)

data_qas_id = DatasetDict({"train": train_dataset, "validation": validation_dataset})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset idk_mrc downloaded and prepared to /root/.cache/huggingface/datasets/idk_mrc/idk_mrc_source/1.0.0/cf468d86fa7341e69998db1449851672ebfb4fa46036929d66b9de15c421334f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████| 3659/3659 [00:16<00:00, 225.65it/s]


In [31]:
def rindex(lst, value, operator=operator):
      return len(lst) - operator.indexOf(reversed(lst), value) - 1

def preprocess_function_qa(examples, tokenizer, MAX_LENGTH=MAX_LENGTH, STRIDE=STRIDE, rindex=rindex, operator=operator):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    examples["context"] = [c.lstrip() for c in examples["context"]]

    tokenized_examples = tokenizer(
      examples['question'],
      examples['context'],
      truncation=True,
      max_length = MAX_LENGTH,
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
      return_tensors='np'
    )

    tokenized_examples['start_positions'] = []
    tokenized_examples['end_positions'] = []

    for seq_idx in range(len(tokenized_examples['input_ids'])):
        seq_ids = tokenized_examples.sequence_ids(seq_idx)
        offset_mappings = tokenized_examples['offset_mapping'][seq_idx]

        cur_example_idx = tokenized_examples['overflow_to_sample_mapping'][seq_idx]

        #answer = examples['answer'][seq_idx][0]
        answer = examples['answer'][cur_example_idx]
        answer = eval(str(answer))
        #answer_text = answer['text'][0]
        answer_start = answer['answer_start']
        #answer_end = answer_start + len(answer_text)
        answer_end = answer['answer_end']

        context_pos_start = seq_ids.index(1)
        context_pos_end = rindex(seq_ids, 1, operator)

        s = e = 0
        if (offset_mappings[context_pos_start][0] <= answer_start and
            offset_mappings[context_pos_end][1] >= answer_end):
          i = context_pos_start
          while offset_mappings[i][0] < answer_start:
            i += 1
          if offset_mappings[i][0] == answer_start:
            s = i
          else:
            s = i - 1

          j = context_pos_end
          while offset_mappings[j][1] > answer_end:
            j -= 1      
          if offset_mappings[j][1] == answer_end:
            e = j
          else:
            e = j + 1

        tokenized_examples['start_positions'].append(s)
        tokenized_examples['end_positions'].append(e)
    return tokenized_examples

# Mulai tokenisasi dan pre-process

In [32]:
tokenized_data_qas_id = data_qas_id.map(
    preprocess_function_qa,
    batched=True,
    remove_columns=data_qas_id['train'].column_names,
    num_proc=1,
    fn_kwargs={'tokenizer': tokenizer, 'MAX_LENGTH': MAX_LENGTH, 'STRIDE': STRIDE, 'rindex': rindex, 'operator': operator}
)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
tokenized_data_qas_id = tokenized_data_qas_id.remove_columns(["offset_mapping", 
                                            "overflow_to_sample_mapping"])

In [34]:
tokenized_data_qas_id.set_format("torch", columns=["input_ids", "token_type_ids"], output_all_columns=True)

In [35]:
tokenized_data_qas_id_train = Dataset.from_dict(tokenized_data_qas_id["train"][:SAMPLE])
tokenized_data_qas_id_validation = Dataset.from_dict(tokenized_data_qas_id["validation"][:SAMPLE])

# Mendefinisikan argumen (dataops) untuk training nanti

In [36]:
TIME_NOW = str(datetime.now()).replace(":", "-").replace(" ", "_").replace(".", "_")
QA = './results/alur2-idk-mrc'
CHECKPOINT_DIR = f'{QA}-{TIME_NOW}/checkpoint/'
MODEL_DIR = f'{QA}-{TIME_NOW}/model/'
OUTPUT_DIR = f'{QA}-{TIME_NOW}/output/'
ACCURACY_DIR = f'{QA}-{TIME_NOW}/accuracy/'

# Mendefinisikan Training Arguments untuk train

In [37]:
training_args_qa = TrainingArguments(
    
    # Checkpoint
    output_dir=CHECKPOINT_DIR,
    save_strategy='epoch',
    save_total_limit=EPOCH,
    
    # Log
    report_to='tensorboard',
    logging_strategy='steps',
    logging_first_step=True,
    logging_steps=LOGGING_STEPS,
    
    # Train
    num_train_epochs=EPOCH,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_ratio=WARMUP_RATIO,
    bf16=False,
    dataloader_num_workers=cpu_count(),
    
    # Miscellaneous
    evaluation_strategy='epoch',
    seed=SEED,
)

# Pendefinisian model Question Answering

In [38]:
model_qa = BertForQuestionAnswering.from_pretrained(MODEL_NAME)

In [39]:
model_qa = model_qa.to(device)

# Melakukan pengumpulan data dengan padding

In [40]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Mulai training untuk fine-tune SQUAD diatas IndoBERT

In [41]:
import string
import collections

# # Melakukan evaluasi dari prediksi
def normalize_text(s):
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_f1_prec_rec(pred, gold):
    pred_tokens = normalize_text(pred).split() # True positive + False positive = Untuk precision
    gold_tokens = normalize_text(gold).split() # True positive + False negatives = Untuk recall
    common = collections.Counter(pred_tokens) & collections.Counter(gold_tokens)
    num_same = sum(common.values()) # True positive

    if len(gold_tokens) == 0 or len(pred_tokens) == 0: 
        return int(gold_tokens == pred_tokens)

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(gold_tokens)
    f1 = (2.0 * precision * recall) / (precision + recall)

    return f1

def compute_metrics(predict_result):
    predictions_idx = np.argmax(predict_result.predictions, axis=2)
    denominator = len(predictions_idx[0])
    label_array = np.asarray(predict_result.label_ids)
    total_correct = 0
    f1_array = []
    precision_array = []
    recall_array = []

    for i in range(len(predict_result.predictions[0])):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1

        pred_text = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                    [start_pred_idx: end_pred_idx])
        gold_text = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                    [start_gold_idx: end_gold_idx])

        if pred_text == gold_text:
            total_correct += 1

        f1 = compute_f1_prec_rec(pred=pred_text, gold=gold_text)

        f1_array.append(f1)

    exact_match = ((total_correct / denominator) * 100.0)
    final_f1 = np.mean(f1_array) * 100.0

    return {'exact_match': exact_match, 'f1': final_f1}

In [42]:
trainer_qa = Trainer(
    model=model_qa,
    args=training_args_qa,
    #train_dataset=tokenized_data_qas_id_train,
    #eval_dataset=tokenized_data_qas_id_validation,
    tokenizer=tokenizer,
    #data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

In [43]:
#trainer_qa.train()

# Menyimpan model Question Answering

In [44]:
trainer_qa.save_model(MODEL_DIR)

Saving model checkpoint to ./results/alur2-idk-mrc-2023-03-26_10-07-40_084469/model/
Configuration saved in ./results/alur2-idk-mrc-2023-03-26_10-07-40_084469/model/config.json
Model weights saved in ./results/alur2-idk-mrc-2023-03-26_10-07-40_084469/model/pytorch_model.bin
tokenizer config file saved in ./results/alur2-idk-mrc-2023-03-26_10-07-40_084469/model/tokenizer_config.json
Special tokens file saved in ./results/alur2-idk-mrc-2023-03-26_10-07-40_084469/model/special_tokens_map.json


# Melakukan prediksi dari model

In [45]:
predict_result = trainer_qa.predict(tokenized_data_qas_id_validation)
predict_result

***** Running Prediction *****
  Num examples = 764
  Batch size = 8


PredictionOutput(predictions=(array([[-8.514204 , -8.426989 , -8.106972 , ..., -8.405422 , -8.40814  ,
        -9.136951 ],
       [-7.3476777, -7.306528 , -4.8059587, ..., -9.1575775, -9.139073 ,
        -9.153362 ],
       [-7.382496 , -7.642652 , -4.0328264, ..., -8.926677 , -8.9915085,
        -8.875996 ],
       ...,
       [-7.005841 , -7.613452 , -8.198171 , ..., -7.45728  , -7.7524405,
        -6.498203 ],
       [-7.6022134, -7.4058123, -8.244312 , ..., -8.298668 , -8.298182 ,
        -7.5270724],
       [-7.6271653, -7.4068747, -8.218372 , ..., -8.294968 , -8.294479 ,
        -7.605162 ]], dtype=float32), array([[-8.534399 , -8.604107 , -8.806009 , ..., -7.257086 , -7.261894 ,
        -8.723284 ],
       [-6.931199 , -7.8831453, -6.8691025, ..., -8.48402  , -8.358842 ,
        -8.258781 ],
       [-7.578419 , -8.0381155, -6.6029034, ..., -8.459461 , -8.628995 ,
        -8.426313 ],
       ...,
       [-6.358011 , -7.654443 , -7.941966 , ..., -8.342517 , -7.7406406,
        -8

In [46]:
os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)
with open(f'{OUTPUT_DIR}/output.txt', "w") as f:
  f.write(str(predict_result))
  f.close()

# Melakukan evaluasi dari prediksi

In [47]:
metric_result_before_filtering = compute_metrics(predict_result)
metric_result_before_filtering

{'exact_match': 36.518324607329845, 'f1': 42.63542386279981}

In [48]:
os.makedirs(os.path.dirname(ACCURACY_DIR), exist_ok=True)
with open(f'{ACCURACY_DIR}/accuracy.txt', "w") as f:
  f.write(str(metric_result_before_filtering))
  f.close()

## Coba Alur 2

In [49]:
from tqdm import tqdm

def create_qas_dataframe(predict_result=predict_result, index_largest=1):
    predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, index_largest * -1]
    #predictions_idx = np.argmax(predict_result.predictions, axis=2)
    label_array = np.asarray(predict_result.label_ids)
    question_decoded = []
    context_decoded = []
    pred_answer_decoded = []
    gold_answer_decoded = []
    
    for i in tqdm(range(len(predict_result.predictions[0]))):
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        pred_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_pred_idx: end_pred_idx], skip_special_tokens=True)
        pred_answer_decoded.append(pred_answer)
        
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1
        gold_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_gold_idx: end_gold_idx], skip_special_tokens=True)
        gold_answer_decoded.append(gold_answer)
        
        question = []
        context = []
         
        for j in range(len(tokenized_data_qas_id_validation[i]['token_type_ids'])):
            if tokenized_data_qas_id_validation[i]['token_type_ids'][j] == 0:
                question.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
            else:
                context.append(tokenized_data_qas_id_validation[i]['input_ids'][j])

        question_decoded.append(tokenizer.decode(question, skip_special_tokens=True))
        context_decoded.append(tokenizer.decode(context, skip_special_tokens=True))
    
    qas_df = pd.DataFrame({'Context': context_decoded, 
                           'Question': question_decoded, 
                           'Prediction Answer': pred_answer_decoded,
                          'Gold Answer': gold_answer_decoded})
                      
    return qas_df

In [50]:
qas_df = create_qas_dataframe(predict_result)
qas_df

100%|█████████████████████████████████████████████████████████████████████████████████| 764/764 [05:00<00:00,  2.54it/s]


,Context,Question,Prediction Answer,Gold Answer
0,sistem pemosisi global [ 1 ] ( bahasa inggris ...,apa kepanjangan dari gps?,global positioning system,global positioning system
1,"ukuran reptil bervariasi, dari yang berukuran ...",apakah cabang ilmu pengetahuan alam yang mempe...,herpetologi,herpetologi
2,"ukuran reptil bervariasi, dari yang berukuran ...",apa cabang ilmu pengetahuan alam yang tidak me...,herpetologi,
3,"reptil ( binatang melata, atau dalam bahasa la...",apakah maksud reptil dalam bahasa latin?,kelompok hewan vertebrata berdarah dingin dan ...,' melata'atau'merayap '
4,"reptil ( binatang melata, atau dalam bahasa la...",apakah maksud reptil ganas dalam bahasa latin?,kelompok hewan vertebrata berdarah dingin dan ...,
...,...,...,...,...
759,"realme meluncurkan smartphone pertamanya "" rea...",apakah smartphone pertama yang diproduksi realme?,,realme 1
760,thariq bin ziyad lebih banyak dikenal sebagai ...,siapa yang dikenal sebagai penakluk spanyol?,thariq bin ziyad,thariq bin ziyad
761,thariq bin ziyad lebih banyak dikenal sebagai ...,siapa yang dikenal sebagai penakluk spanyol pa...,thariq bin ziyad,
762,adalah angkatan laut kekaisaran jepang ( kaigu...,siapa yang menggunakan kapal induk secara efek...,angkatan laut kekaisaran jepang,angkatan laut kekaisaran jepang ( kaigun )


In [51]:
question_mark = ['siapa', 'siapakah',
                    'apa', 'apakah', 'adakah',
                    'dimana', 'dimanakah', 'darimanakah',
                    'kapan', 'kapankah',
                    'bagaimana', 'bagaimanakah',
                    'kenapa', 'mengapa',
                    'berapa', 'berapakah', 'seberapa']

## Mencoba cara retrieve model dari HF

In [52]:
!pip install torch torchvision torchaudio

  Using cached torch-2.0.0-cp38-cp38-manylinux1_x86_64.whl (619.9 MB)
  Using cached torchaudio-2.0.1-cp38-cp38-manylinux1_x86_64.whl (4.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1
    Uninstalling torchaudio-0.13.1:
      Successfully uninstalled torchaudio-0.13.1


In [53]:
torch.cuda.is_available()

False

In [54]:
!nvidia-smi

Sun Mar 26 10:14:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   36C    P0    57W / 300W |  32476MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   60C    P0   289W / 300W |  11709MiB / 32480MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [55]:
from transformers import pipeline

pretrained_name_sc = "afaji/fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased"
pretrained_name_qa = "afaji/fine-tuned-DatasetQAS-TYDI-QA-ID-with-indobert-base-uncased-with-ITTL-without-freeze-LR-1e-05"
tokenizer_kwargs = {'padding': True, 'truncation': True, 'max_length': MAX_LENGTH}

nlp_sc = pipeline(task="text-classification", model=pretrained_name_sc, tokenizer=pretrained_name_sc, 
                   **tokenizer_kwargs)
nlp_qa = pipeline(task="question-answering", model=pretrained_name_qa, tokenizer=pretrained_name_qa, 
                   **tokenizer_kwargs)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--afaji--fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased/snapshots/baf8065c541ffd323cf43d1e93868ccbb20febbd/config.json
Model config BertConfig {
  "_name_or_path": "afaji/fine-tuned-IndoNLI-Augmented-with-indobert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "positi

In [56]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

pretrained_name_tg = "Wikidepia/IndoT5-base-paraphrase"

tokenizer_kwargs = {'truncation': True, 'max_length': MAX_LENGTH}

nlp_tg = pipeline(task="text2text-generation", model=pretrained_name_tg, tokenizer=pretrained_name_tg, 
                  )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Wikidepia--IndoT5-base-paraphrase/snapshots/5d591dc3aeae0aade0f327a5ebdd0f071c83f567/config.json
Model config T5Config {
  "_name_or_path": "Wikidepia/IndoT5-base-paraphrase",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "v

## Coba buat method evaluasi

In [57]:
def smoothing(question, pred_answer, gold_answer, type, question_mark=question_mark):
    
    if type == 'replace first':
        pred_hypothesis = question.replace('?', '')
        pred_hypothesis = pred_hypothesis.replace(question.split()[0], pred_answer)

        gold_hypothesis = question.replace('?', '')
        gold_hypothesis = gold_hypothesis.replace(question.split()[0], gold_answer)
    
    elif type == 'replace question mark':
        for i in question.split():
            if i in question_mark:
                pred_hypothesis = question.replace('?', '')
                pred_hypothesis = pred_hypothesis.replace(i, pred_answer)

                gold_hypothesis = question.replace('?', '')
                gold_hypothesis = gold_hypothesis.replace(i, gold_answer)
    
    elif type == 'add adalah':
        pred_hypothesis = question.replace('?', '')
        pred_hypothesis = pred_hypothesis.replace(question.split()[0], '')
        pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

        gold_hypothesis = question.replace('?', '')
        gold_hypothesis = gold_hypothesis.replace(question.split()[0], '')
        gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"
    
    elif type == 'just concat answer and question':
        pred_hypothesis = f"{question} {pred_answer}"         
        gold_hypothesis = f"{question} {gold_answer}"
        
    elif type == 'rule based':
        question = question.replace('kah', '')
        for j in question.split():
            if j in question_mark:
                if j == 'siapa' or j == 'siapakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"

                elif j == 'apa' or j == 'apakah' or j == 'adakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"

                elif j == 'dimana' or j == 'dimanakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} di {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} di {gold_answer}"

                elif j == 'darimanakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} dari {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} dari {gold_answer}"

                elif j == 'kapan' or j == 'kapankah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} pada {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} pada {gold_answer}"

                elif j == 'bagaimana' or j == 'bagaimanakah':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '')
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"

                elif j == 'kenapa' or j == 'mengapa':
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, 'alasan').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah karena {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, 'alasan').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah karena {gold_answer}"

                elif j == 'berapa' or j == 'berapakah' or j == 'seberapa': 
                    pred_hypothesis = question.replace('?', '')
                    pred_hypothesis = pred_hypothesis.replace(j, '').lstrip()
                    pred_hypothesis = f"{pred_hypothesis} adalah {pred_answer}"

                    gold_hypothesis = question.replace('?', '')
                    gold_hypothesis = gold_hypothesis.replace(j, '').lstrip()
                    gold_hypothesis = f"{gold_hypothesis} adalah {gold_answer}"
                    
    elif type == 'machine generation': 
        pred_hypothesis, gold_hypothesis = smoothing(question, pred_answer, gold_answer, type="rule based")
        pred_hypothesis = nlp_tg(pred_hypothesis)[0]['generated_text']
        gold_hypothesis = nlp_tg(gold_hypothesis)[0]['generated_text']
        
    return pred_hypothesis, gold_hypothesis

In [58]:
pred_hypothesis, gold_hypothesis = smoothing("kenapa messi menang?", "emang jago kali", "tim yang unggul",
                                            type="machine generation")
print(pred_hypothesis)
print(gold_hypothesis)

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



alasan mengapa Messi menang adalah karena dia jago kali
alasan messi menang adalah karena tim yang unggul .


In [125]:
def create_df_for_evaluation(predict_result, type_smoothing, type_qas, MAXIMUM_SEARCH_ITER=MAXIMUM_SEARCH_ITER):
    
    # Ekstrak dari PredictionOutput QAS
    predictions_idx = np.argsort(predict_result.predictions, axis=2)[:, :, 1 * -1]
    label_array = np.asarray(predict_result.label_ids)
    
    question_array = []
    context_array = []
    
    pred_answer_before_filtering_array = []
    pred_answer_after_filtering_array = []
    
    label_before_filtering_array = []
    label_after_filtering_array = []
    
    pred_hypothesis_before_filtering_array = []
    pred_hypothesis_after_filtering_array = []
    
    gold_answer_array = []
    gold_hypothesis_array = []
    
    # Iterasi ini ditujukan untuk retrieve answer
    for i in tqdm(range(len(predict_result.predictions[0]))):
        
        isFoundBiggest = False
        
        start_pred_idx = predictions_idx[0][i]
        end_pred_idx = predictions_idx[1][i] + 1
        
        start_gold_idx = label_array[0][i]
        end_gold_idx = label_array[1][i] + 1
        
        # Retrieve answer prediksi
        pred_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_pred_idx: end_pred_idx], skip_special_tokens=True)
        
        # Retrieve answer gold
        gold_answer = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                       [start_gold_idx: end_gold_idx], skip_special_tokens=True)
        
        question = []
        context = []
        
        # Iterasi ini untuk retrieve question dan context index yang bersangkutan
        for j in range(len(tokenized_data_qas_id_validation[i]['token_type_ids'])):
            
            # Bila token_type_ids-nya 0, maka itu question (sesuai dengan urutan tokenisasi)
            if tokenized_data_qas_id_validation[i]['token_type_ids'][j] == 0:
                question.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
            
            # Bila token_type_ids-nya 1, maka itu context (sesuai dengan urutan tokenisasi)
            else:
                context.append(tokenized_data_qas_id_validation[i]['input_ids'][j])
        
        question_decoded = tokenizer.decode(question, skip_special_tokens=True)
        context_decoded = tokenizer.decode(context, skip_special_tokens=True)
        pred_hypothesis, gold_hypothesis = smoothing(question_decoded, pred_answer, gold_answer, type_smoothing)

        # Cek label dari answer prediksi dan context
        predicted_label = nlp_sc({'text': context_decoded, 
                                  'text_pair': pred_hypothesis}, 
                                 **tokenizer_kwargs)
        
        pred_answer_before_filtering_array.append([pred_answer])
        pred_hypothesis_before_filtering_array.append([pred_hypothesis])
        label_before_filtering_array.append([predicted_label])
        
        # Cek label dari answer prediksi dan context, bila labelnya entailment (atau neutral), maka answernya jadi hasil akhir
        if predicted_label['label'] == 'neutral':
            if type_qas == 'entailment or neutral':
                question_array.append(question_decoded)
                context_array.append(context_decoded)
                pred_answer_after_filtering_array.append([pred_answer])
                gold_answer_array.append(gold_answer)
                pred_hypothesis_after_filtering_array.append([pred_hypothesis])
                gold_hypothesis_array.append(gold_hypothesis)
                label_after_filtering_array.append([predicted_label])

        if predicted_label['label'] == 'entailment':
            if type_qas == 'entailment only' or type_qas == 'entailment or neutral':
                question_array.append(question_decoded)
                context_array.append(context_decoded)
                pred_answer_after_filtering_array.append([pred_answer])
                gold_answer_array.append(gold_answer)
                pred_hypothesis_after_filtering_array.append([pred_hypothesis])
                gold_hypothesis_array.append(gold_hypothesis)
                label_after_filtering_array.append([predicted_label])
            
        # Cek label dari answer prediksi dan context, bila labelnya bukan entailment (atau neutral), 
        # -- maka masuk ke for-loop untuk iterasi ke argmax selanjutnya, dengan menggunakan argsort
        else:
            
            if predicted_label == 'neutral' and type_qas == 'entailment or neutral': continue
            
            # Bila MAXIMUM_SEARCH_ITER dibawah 2, maka continue langsung
            if MAXIMUM_SEARCH_ITER < 2: continue

            # Bila MAXIMUM_SEARCH_ITER diatas 2, maka continue langsung
            
            else:
                # Bila bukan entailment, loop sebanyak MAXIMUM_SEARCH_ITER kali.
                pred_answer_after_filtering_array_msi_recorded = []
                pred_hypothesis_after_filtering_array_msi_recorded = []
                label_after_filtering_array_msi_recorded = []
                for index_largest in range(MAXIMUM_SEARCH_ITER - 1):
                    
                    #pred_answer_after_filtering_array_msi_recorded = []
                    #pred_hypothesis_after_filtering_array_msi_recorded = []
                    #label_after_filtering_array_msi_recorded = []

                    # Cari di index kedua, ketiga, keempat, dan seterusnya
                    predictions_idx_inside_loop = np.argsort(predict_result.predictions, 
                                                             axis=2)[:, :, (index_largest + 2) * -1]

                    start_pred_idx = predictions_idx_inside_loop[0][i]
                    end_pred_idx = predictions_idx_inside_loop[1][i] + 1

                    # Retrieve answer prediksi
                    pred_answer_inside_loop = tokenizer.decode(tokenized_data_qas_id_validation[i]['input_ids']
                                                   [start_pred_idx: end_pred_idx], skip_special_tokens=True)
                    
                    pred_hypothesis_inside_loop, gold_hypothesis = smoothing(
                        question_decoded, pred_answer_inside_loop, gold_answer, type_smoothing)
                    
                    # Cek label dari answer prediksi dan context
                    predicted_label_inside_loop = nlp_sc({'text': context_decoded, 
                                                          'text_pair': pred_hypothesis_inside_loop}
                                                           , **tokenizer_kwargs)
                    
                    pred_answer_after_filtering_array_msi_recorded.append(pred_answer_inside_loop)
                    pred_hypothesis_after_filtering_array_msi_recorded.append(pred_hypothesis_inside_loop)
                    label_after_filtering_array_msi_recorded.append(predicted_label_inside_loop)
                    
                    # Bila label-nya sudah entailment (atau neutral), maka answernya jadi hasil akhir, dan break
                    if type_qas == 'entailment only':
                        if predicted_label_inside_loop['label'] == 'entailment':
                            isFoundBiggest = True
                            question_array.append(question_decoded)
                            context_array.append(context_decoded)
                            gold_answer_array.append(gold_answer)   
                            gold_hypothesis_array.append(gold_hypothesis)
                            
                            pred_answer_after_filtering_array.append(pred_answer_after_filtering_array_msi_recorded)
                            pred_hypothesis_after_filtering_array.append(pred_hypothesis_after_filtering_array_msi_recorded)
                            label_after_filtering_array.append(label_after_filtering_array_msi_recorded)
                            break
                            
                    elif type_qas == 'entailment or neutral':
                        if predicted_label_inside_loop['label'] == 'entailment' or predicted_label_inside_loop['label'] == 'neutral':
                            isFoundBiggest = True
                            question_array.append(question_decoded)
                            context_array.append(context_decoded)
                            gold_answer_array.append(gold_answer)   
                            gold_hypothesis_array.append(gold_hypothesis)
                            
                            pred_answer_after_filtering_array.append(pred_answer_after_filtering_array_msi_recorded)
                            pred_hypothesis_after_filtering_array.append(pred_hypothesis_after_filtering_array_msi_recorded)
                            label_after_filtering_array.append(label_after_filtering_array_msi_recorded)
                            break

                if isFoundBiggest == False:
                    # Bila sampai iterasi terakhir, belum entailment (atau neutral) juga, maka append saja jawaban kosong
                    
                    pred_answer_not_found_biggest = "" # Disini, jawaban kosong
                    
                    question_array.append(question_decoded)
                    context_array.append(context_decoded)
                    
                    pred_hypothesis_not_found_biggest, gold_hypothesis = smoothing(
                        question_decoded, pred_answer_not_found_biggest, gold_answer, type_smoothing)
                    
                    pred_answer_after_filtering_array_msi_recorded.append(pred_answer_not_found_biggest)
                    pred_hypothesis_after_filtering_array_msi_recorded.append(pred_hypothesis_not_found_biggest)
                    label_after_filtering_array_msi_recorded.append(predicted_label_inside_loop)
                    
                    gold_answer_array.append(gold_answer)
                    gold_hypothesis_array.append(gold_hypothesis)
                    
                    pred_answer_after_filtering_array.append(pred_answer_after_filtering_array_msi_recorded)
                    pred_hypothesis_after_filtering_array.append(pred_hypothesis_after_filtering_array_msi_recorded)
                    label_after_filtering_array.append(label_after_filtering_array_msi_recorded)
    
    # Buat DataFrame QAS
    qas_df = pd.DataFrame({'Context': context_array, 
                           'Question': question_array, 
                           
                           'Prediction Answer Before Filtering': pred_answer_before_filtering_array,
                           'Prediction Hypothesis Before Filtering': pred_hypothesis_before_filtering_array,
                           'Label Before Filtering': label_before_filtering_array,
                                 
                           'Prediction Answer After Filtering': pred_answer_after_filtering_array,
                           'Prediction Hypothesis After Filtering': pred_hypothesis_after_filtering_array,
                           'Label After Filtering': label_after_filtering_array,
                          
                           'Gold Answer': gold_answer_array,
                          'Gold Hypothesis': gold_hypothesis_array})
                          
    assert len(predict_result.predictions[0]) == len(qas_df), "Jumlah prediksi berbeda dengan jumlah evaluasi"
    
    # Return DataFrame QAS
    return qas_df

In [126]:
eval_df = create_df_for_evaluation(predict_result, type_smoothing='replace first', type_qas='entailment only', MAXIMUM_SEARCH_ITER=2)
eval_df

100%|█████████████████████████████████████████████████████████████████████████████████| 764/764 [06:03<00:00,  2.10it/s]


,Context,Question,Prediction Answer Before Filtering,Prediction Hypothesis Before Filtering,Label Before Filtering,Prediction Answer After Filtering,Prediction Hypothesis After Filtering,Label After Filtering,Gold Answer,Gold Hypothesis
0,sistem pemosisi global [ 1 ] ( bahasa inggris ...,apa kepanjangan dari gps?,[global positioning system],[global positioning system kepanjangan dari gps],"[{'label': 'entailment', 'score': 0.9869425296...",[global positioning system],[global positioning system kepanjangan dari gps],"[{'label': 'entailment', 'score': 0.9869425296...",global positioning system,global positioning system kepanjangan dari gps
1,"ukuran reptil bervariasi, dari yang berukuran ...",apakah cabang ilmu pengetahuan alam yang mempe...,[herpetologi],[herpetologi cabang ilmu pengetahuan alam yang...,"[{'label': 'entailment', 'score': 0.9994400143...",[herpetologi],[herpetologi cabang ilmu pengetahuan alam yang...,"[{'label': 'entailment', 'score': 0.9994400143...",herpetologi,herpetologi cabang ilmu pengetahuan alam yang ...
2,"ukuran reptil bervariasi, dari yang berukuran ...",apa cabang ilmu pengetahuan alam yang tidak me...,[herpetologi],[herpetologi cabang ilmu pengetahuan alam yang...,"[{'label': 'contradiction', 'score': 0.9999427...","[, ]",[ cabang ilmu pengetahuan alam yang tidak memp...,"[{'label': 'contradiction', 'score': 0.9994680...",,cabang ilmu pengetahuan alam yang tidak mempe...
3,"reptil ( binatang melata, atau dalam bahasa la...",apakah maksud reptil dalam bahasa latin?,[kelompok hewan vertebrata berdarah dingin dan...,[kelompok hewan vertebrata berdarah dingin dan...,"[{'label': 'entailment', 'score': 0.9996963739...",[kelompok hewan vertebrata berdarah dingin dan...,[kelompok hewan vertebrata berdarah dingin dan...,"[{'label': 'entailment', 'score': 0.9996963739...",' melata'atau'merayap ',' melata'atau'merayap ' maksud reptil dalam ba...
4,"reptil ( binatang melata, atau dalam bahasa la...",apakah maksud reptil ganas dalam bahasa latin?,[kelompok hewan vertebrata berdarah dingin dan...,[kelompok hewan vertebrata berdarah dingin dan...,"[{'label': 'entailment', 'score': 0.9815422892...",[kelompok hewan vertebrata berdarah dingin dan...,[kelompok hewan vertebrata berdarah dingin dan...,"[{'label': 'entailment', 'score': 0.9815422892...",,maksud reptil ganas dalam bahasa latin
...,...,...,...,...,...,...,...,...,...,...
759,"realme meluncurkan smartphone pertamanya "" rea...",apakah smartphone pertama yang diproduksi realme?,[],[ smartphone pertama yang diproduksi realme],"[{'label': 'entailment', 'score': 0.9927445650...",[],[ smartphone pertama yang diproduksi realme],"[{'label': 'entailment', 'score': 0.9927445650...",realme 1,realme 1 smartphone pertama yang diproduksi re...
760,thariq bin ziyad lebih banyak dikenal sebagai ...,siapa yang dikenal sebagai penakluk spanyol?,[thariq bin ziyad],[thariq bin ziyad yang dikenal sebagai penaklu...,"[{'label': 'entailment', 'score': 0.9913252592...",[thariq bin ziyad],[thariq bin ziyad yang dikenal sebagai penaklu...,"[{'label': 'entailment', 'score': 0.9913252592...",thariq bin ziyad,thariq bin ziyad yang dikenal sebagai penakluk...
761,thariq bin ziyad lebih banyak dikenal sebagai ...,siapa yang dikenal sebagai penakluk spanyol pa...,[thariq bin ziyad],[thariq bin ziyad yang dikenal sebagai penaklu...,"[{'label': 'neutral', 'score': 0.9973908066749...",[##yad lebih banyak dikenal sebagai penakluk s...,[##yad lebih banyak dikenal sebagai penakluk s...,"[{'label': 'contradiction', 'score': 0.8795180...",,yang dikenal sebagai penakluk spanyol paling ...
762,adalah angkatan laut kekaisaran jepang ( kaigu...,siapa yang menggunakan kapal induk secara efek...,[angkatan laut kekaisaran jepang],[angkatan laut kekaisaran jepang yang mengguna...,"[{'label': 'entailment', 'score': 0.9997662901...",[angkatan laut kekaisaran jepang],[angkatan laut kekaisaran jepang yang mengguna...,"[{'label': 'entailment', 'score': 0.9997662901...",angkatan laut kekaisaran jepang ( kaig

In [119]:
eval_df['Question'][2]

'apa cabang ilmu pengetahuan alam yang tidak mempelajari tentang reptil?'

In [120]:
eval_df['Context'][2]

'ukuran reptil bervariasi, dari yang berukuran hingga 1, 6cm ( tokek kecil, sphaerodactylus ariasae ) hingga berukuran 6 m dan mencapai berat 1 ton ( buaya air asin, crocodylus porosus ). cabang ilmu pengetahuan alam yang mempelajari tentang reptil adalah herpetologi.'

In [131]:
eval_df['Prediction Answer After Filtering'][2]

['', '']

In [62]:
def compute_metrics_from_df(df, type_qas):
    
    denominator = len(df)
    total_correct = 0
    f1_array = []
    
    true_positive_before_filtering = 0
    false_positive_before_filtering = 0
    false_negative_before_filtering = 0
    true_negative_before_filtering = 0
    
    true_positive_after_filtering = 0
    false_positive_after_filtering = 0
    false_negative_after_filtering = 0
    true_negative_after_filtering = 0

    for i in range(len(df)):
        
        pred_answer_before_filtering = df["Prediction Answer Before Filtering"][i][-1]
        pred_answer_after_filtering = df["Prediction Answer After Filtering"][i][-1]
        
        pred_label_before_filtering = df["Label Before Filtering"][i][-1]['label']
        pred_label_after_filtering = df["Label After Filtering"][i][-1]['label']
        
        gold_text = df["Gold Answer"][i]

        if pred_answer_after_filtering == gold_text:
            total_correct += 1

        f1 = compute_f1_prec_rec(pred=pred_answer_after_filtering, gold=gold_text)

        f1_array.append(f1)
        
        # Terprediksi dengan label yang benar, dan hasil answernya benar -> True positive
        # Terprediksi dengan label yang benar, padahal hasil answernya salah -> False positive
        # Terprediksi dengan label yang salah, padahal hasil answernya benar -> False negative
        # Terprediksi dengan label yang salah, dan hasil answernya salah -> True negative
        
        if type_qas == 'entailment only':
        
            if (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'entailment'):
                true_positive_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'entailment'):
                false_positive_after_filtering += 1
            elif (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering != 'entailment'):
                false_negative_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering != 'entailment'):
                true_negative_after_filtering += 1

            if (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'entailment'):
                true_positive_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'entailment'):
                false_positive_before_filtering += 1
            elif (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering != 'entailment'):
                false_negative_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering != 'entailment'):
                true_negative_before_filtering += 1
        
        elif type_qas == 'entailment or neutral':
        
            if (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering == 'entailment' 
                                                               or pred_label_after_filtering == 'neutral'):
                true_positive_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering == 'entailment' 
                                                                 or pred_label_after_filtering == 'neutral'):
                false_positive_after_filtering += 1
            elif (pred_answer_after_filtering == gold_text) and (pred_label_after_filtering != 'entailment' 
                                                                 and pred_label_after_filtering != 'neutral'):
                false_negative_after_filtering += 1
            elif (pred_answer_after_filtering != gold_text) and (pred_label_after_filtering != 'entailment' 
                                                                 and pred_label_after_filtering != 'neutral'):
                true_negative_after_filtering += 1

            if (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'entailment' 
                                                                or pred_label_after_filtering == 'neutral'):
                true_positive_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'entailment' 
                                                                  or pred_label_after_filtering == 'neutral'):
                false_positive_before_filtering += 1
            elif (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering != 'entailment' 
                                                                  and pred_label_after_filtering != 'neutral'):
                false_negative_before_filtering += 1
            elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering != 'entailment' 
                                                                  and pred_label_after_filtering != 'neutral'):
                true_negative_before_filtering += 1

    exact_match = ((total_correct / denominator) * 100.0)
    final_f1 = np.mean(f1_array) * 100.0
    after_filtering_metric_array = [true_positive_after_filtering, false_positive_after_filtering, 
                          false_negative_after_filtering, true_negative_after_filtering]
    before_filtering_metric_array = [true_positive_before_filtering, false_positive_before_filtering, 
                          false_negative_before_filtering, true_negative_before_filtering]

    return {'exact_match': exact_match, 'f1': final_f1}, after_filtering_metric_array, before_filtering_metric_array

In [63]:
metric_result_after_filtering, after_filtering_metric_array, before_filtering_metric_array = compute_metrics_from_df(
    eval_df, "entailment only")
metric_result_after_filtering

{'exact_match': 62.17277486910995, 'f1': 66.467874209069}

In [64]:
def convert_to_non_zero(number):
    if number == 0:
        number += sys.float_info.min
    return number

def compute_f1_prec_rec_whole(metric_array):
    accuracy = (metric_array[0] + metric_array[3]) / \
        (metric_array[0] + metric_array[1] + 
         metric_array[2] + metric_array[3])
    
    precision = (metric_array[0]) / (metric_array[0] + metric_array[1])
    
    recall = (metric_array[0]) / (metric_array[0] + metric_array[2])
    
    f1 = (2 * precision * recall) / (precision + recall)
    
    return accuracy, precision, recall, f1

def diff_verbose_metric(metric_result_before, metric_result_after, metric):
    
    percentage = round(((metric_result_after - metric_result_before) / metric_result_before) * 100, 2)
    
    if '&' in metric: vocab = "nilai"
    else: vocab = "metrik"
    
    if metric_result_before ==  metric_result_after:
        print(f"Hasil {vocab} {metric} sebelum filtering NLI SAMA DENGAN metrik setelah filtering NLI")
    elif metric_result_before <  metric_result_after:
        print(f"Hasil {vocab} {metric} setelah filtering NLI mengalami KENAIKAN sebesar: {percentage} %")
    elif metric_result_before >  metric_result_after:
        print(f"Hasil {vocab} {metric} setelah filtering NLI mengalami PENURUNAN sebesar: {-1 * percentage} %")
    
    return percentage

In [65]:
def compare_metrics(metrics_before, metrics_after, 
                    after_filtering_metric_array=after_filtering_metric_array, 
                    before_filtering_metric_array=before_filtering_metric_array):
    
    em_before = metrics_before['exact_match']
    f1_before = metrics_before['f1']

    print("~ METRIK PER TOKEN ~")
    print(f"Skor Exact Match sebelum filtering NLI: {em_before}")
    print(f"Skor F1 sebelum filtering NLI: {f1_before}")
    print()

    em_after = metrics_after['exact_match']
    f1_after = metrics_after['f1']

    print(f"Skor Exact Match setelah filtering NLI: {em_after}")
    print(f"Skor F1 setelah filtering NLI: {f1_after}")
    print()

    em_before = convert_to_non_zero(em_before)
    f1_before = convert_to_non_zero(f1_before)

    em_after = convert_to_non_zero(em_after)
    f1_after = convert_to_non_zero(f1_after)

    print("~ METRIK DENGAN PARAMETER NLI ~")
    print(f"[BEFORE FILTERING] Jawaban benar & label NLI yang sesuai: {before_filtering_metric_array[0]}")
    print(f"[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: {before_filtering_metric_array[1]}")
    print(f"[BEFORE FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: {before_filtering_metric_array[2]}")
    print(f"[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: {before_filtering_metric_array[3]}")
    print()

    print(f"[AFTER FILTERING] Jawaban benar & label NLI yang sesuai: {after_filtering_metric_array[0]}")
    print(f"[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: {after_filtering_metric_array[1]}")
    print(f"[AFTER FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: {after_filtering_metric_array[2]}")
    print(f"[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: {after_filtering_metric_array[3]}")
    print()

    print("Metrik di atas, bisa direpresentasikan menjadi:")

    acc_before_whole, prec_before_whole, rec_before_whole, f1_before_whole = compute_f1_prec_rec_whole(
        before_filtering_metric_array)
    acc_after_whole, prec_after_whole, rec_after_whole, f1_after_whole = compute_f1_prec_rec_whole(
        after_filtering_metric_array)

    print(f"[BEFORE FILTERING] Akurasi: {acc_before_whole}")
    print(f"[BEFORE FILTERING] Precision: {prec_before_whole}")
    print(f"[BEFORE FILTERING] Recall: {rec_before_whole}")
    print(f"[BEFORE FILTERING] F1: {f1_before_whole}")
    print()

    print(f"[AFTER FILTERING] Akurasi: {acc_after_whole}")
    print(f"[AFTER FILTERING] Precision: {prec_after_whole}")
    print(f"[AFTER FILTERING] Recall: {rec_after_whole}")
    print(f"[AFTER FILTERING] F1: {f1_after_whole}")
    print()

    print("--- Persentase perubahan hasil metrik ---")
    print("~ METRIK PER TOKEN ~")
    diff_verbose_metric(em_before, em_after, "Exact Match")
    diff_verbose_metric(f1_before, f1_after, "F1")
    print()

    print("~ METRIK DENGAN PARAMETER NLI ~")
    diff_verbose_metric(before_filtering_metric_array[0], after_filtering_metric_array[0], 
                        "Jawaban benar & label NLI yang sesuai")
    diff_verbose_metric(before_filtering_metric_array[1], after_filtering_metric_array[1], 
                        "Jawaban TIDAK benar & label NLI yang sesuai")
    diff_verbose_metric(before_filtering_metric_array[2], after_filtering_metric_array[2], 
                        "Jawaban benar & label NLI yang TIDAK sesuai")
    diff_verbose_metric(before_filtering_metric_array[3], after_filtering_metric_array[3], 
                        "Jawaban TIDAK benar & label NLI yang TIDAK sesuai")
    print()

    print("Metrik di atas, bisa direpresentasikan menjadi:")
    diff_verbose_metric(acc_before_whole, acc_after_whole, "Akurasi")
    diff_verbose_metric(prec_before_whole, prec_after_whole, "Precision")
    diff_verbose_metric(rec_before_whole, rec_after_whole, "Recall")
    diff_verbose_metric(f1_before_whole, f1_after_whole, "F1")
    print()

In [66]:
compare_metrics(metric_result_before_filtering, metric_result_after_filtering)

~ METRIK PER TOKEN ~
Skor Exact Match sebelum filtering NLI: 36.518324607329845
Skor F1 sebelum filtering NLI: 42.63542386279981

Skor Exact Match setelah filtering NLI: 62.17277486910995
Skor F1 setelah filtering NLI: 66.467874209069

~ METRIK DENGAN PARAMETER NLI ~
[BEFORE FILTERING] Jawaban benar & label NLI yang sesuai: 199
[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: 185
[BEFORE FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: 80
[BEFORE FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: 300

[AFTER FILTERING] Jawaban benar & label NLI yang sesuai: 199
[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang sesuai: 185
[AFTER FILTERING] Jawaban benar & label NLI yang TIDAK sesuai: 276
[AFTER FILTERING] Jawaban TIDAK benar & label NLI yang TIDAK sesuai: 104

Metrik di atas, bisa direpresentasikan menjadi:
[BEFORE FILTERING] Akurasi: 0.6531413612565445
[BEFORE FILTERING] Precision: 0.5182291666666666
[BEFORE FILTERING] Recall: 0.7132616487455197
[

In [67]:
os.makedirs(os.path.dirname(ACCURACY_DIR), exist_ok=True)
with open(f'{ACCURACY_DIR}/metric_comparison_results.txt', "w") as f, contextlib.redirect_stdout(f):
    compare_metrics(metric_result_before_filtering, metric_result_after_filtering)
    f.close()

In [234]:
def breakdown_evaluation(df, TYPE_QAS, MAXIMUM_SEARCH_ITER=MAXIMUM_SEARCH_ITER):
    
    exist_true_answer_label_entailment = 0
    exist_true_answer_label_neutral = 0
    exist_true_answer_label_contradiction = 0
    
    exist_false_answer_label_entailment = 0
    exist_false_answer_label_neutral = 0
    exist_false_answer_label_contradiction = 0
    
    no_exist_true_answer = 0
    no_exist_false_answer = 0
    
    filtered_in_right_answer_to_filtered_out_wrong_answer = 0
    filtered_out_wrong_answer_to_filtered_in_right_answer = 0
    
    filtered_in_right_answer_to_filtered_out_wrong_answer_unanswered = 0
    filtered_out_wrong_answer_to_filtered_in_right_answer_unanswered = 0
    
    for i in range(len(df)):
        
        pred_answer_before_filtering = df["Prediction Answer Before Filtering"][i][-1]
        pred_answer_after_filtering = df["Prediction Answer After Filtering"][i][-1]
        
        pred_label_before_filtering = df["Label Before Filtering"][i][-1]['label']
        pred_label_after_filtering = df["Label After Filtering"][i][-1]['label']
        
        pred_prob_dist_before_filtering = df["Label Before Filtering"][i][-1]['score']
        pred_prob_dist_after_filtering = df["Label After Filtering"][i][-1]['score']
        
        gold_text = df["Gold Answer"][i]
        
        if (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'entailment') and (pred_answer_before_filtering != ""):
            exist_true_answer_label_entailment += 1
        elif (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'neutral') and (pred_answer_before_filtering != ""):
            exist_true_answer_label_neutral += 1
            if (TYPE_QAS == 'entailment only'):
                if (pred_answer_after_filtering != gold_text):
                    filtered_in_right_answer_to_filtered_out_wrong_answer += 1
                #print(f"Prob Distribution right answer but wrong label NLI before filtering: {pred_prob_dist_before_filtering}")
                #print(f"Prob Distribution right answer but wrong label NLI after filtering: {pred_prob_dist_after_filtering}")
                #print()
                
        elif (pred_answer_before_filtering == gold_text) and (pred_label_before_filtering == 'contradiction') and (pred_answer_before_filtering != ""):
            exist_true_answer_label_contradiction += 1
            if (pred_answer_after_filtering != gold_text):
                filtered_in_right_answer_to_filtered_out_wrong_answer += 1
            #print(f"Prob Distribution right answer but wrong label NLI before filtering: {pred_prob_dist_before_filtering}")
            #print(f"Prob Distribution right answer but wrong label NLI after filtering: {pred_prob_dist_after_filtering}")
            #print()
        
        elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'entailment') and (pred_answer_before_filtering != ""):
            exist_false_answer_label_entailment += 1
        elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'neutral') and (pred_answer_before_filtering != ""):
            exist_false_answer_label_neutral += 1
            if (TYPE_QAS == 'entailment only'):
                if (pred_answer_after_filtering == gold_text):
                    filtered_out_wrong_answer_to_filtered_in_right_answer += 1
        elif (pred_answer_before_filtering != gold_text) and (pred_label_before_filtering == 'contradiction') and (pred_answer_before_filtering != ""):
            exist_false_answer_label_contradiction += 1
            if (pred_answer_after_filtering == gold_text):
                filtered_out_wrong_answer_to_filtered_in_right_answer += 1
        
        elif (pred_answer_before_filtering == gold_text) and (pred_answer_before_filtering == ""):
            no_exist_true_answer += 1
            if (pred_answer_after_filtering != gold_text):
                filtered_in_right_answer_to_filtered_out_wrong_answer_unanswered += 1
        elif (pred_answer_before_filtering != gold_text) and (pred_answer_before_filtering == ""):
            no_exist_false_answer += 1
            if (pred_answer_after_filtering == gold_text):
                filtered_out_wrong_answer_to_filtered_in_right_answer_unanswered += 1
            
        #print(f"Pred answer before filtering: {pred_answer_before_filtering}")
        #print(f"Pred answer after filtering: {pred_answer_after_filtering}")
        #print(f"Gold answer: {gold_text}")
        #print()
    
    print(f"Jawaban benar (answer exist) entailment: {exist_true_answer_label_entailment}")
    print(f"Jawaban benar (answer exist) neutral: {exist_true_answer_label_neutral}")
    print(f"Jawaban benar (answer exist) contradiction: {exist_true_answer_label_contradiction}")
    print()
    print(f"Jawaban salah (answer exist) entailment: {exist_false_answer_label_entailment}")
    print(f"Jawaban salah (answer exist) neutral: {exist_false_answer_label_neutral}")
    print(f"Jawaban salah (answer exist) contradiction: {exist_false_answer_label_contradiction}")
    print()
    print(f"Jawaban prediksi no-answer benar: {no_exist_true_answer}")
    print(f"Jawaban prediksi no-answer salah: {no_exist_false_answer}")
    print()
    print(f"Jawaban prediksi benar, tapi tidak masuk prediksi final (menjadi salah) karena label NLI yang tidak sesuai: {filtered_in_right_answer_to_filtered_out_wrong_answer}")
    print(f"Jawaban prediksi salah, tapi masuk prediksi final (menjadi benar) karena label NLI yang sesuai: {filtered_out_wrong_answer_to_filtered_in_right_answer}")
    print()
    print(f"Jawaban prediksi benar (unanswerable), tapi tidak masuk prediksi final (menjadi salah) karena label NLI yang tidak sesuai: {filtered_in_right_answer_to_filtered_out_wrong_answer_unanswered}")
    print(f"Jawaban prediksi salah (unanswerable), tapi masuk prediksi final (menjadi benar) karena label NLI yang sesuai: {filtered_out_wrong_answer_to_filtered_in_right_answer_unanswered}")
    print()
    print(f"Total prediksi jawaban: {len(df)}")
    print()
    
    assert len(df) == exist_true_answer_label_entailment+exist_true_answer_label_neutral+exist_true_answer_label_contradiction+\
            exist_false_answer_label_entailment+exist_false_answer_label_neutral+exist_false_answer_label_contradiction+\
            no_exist_true_answer+no_exist_false_answer

In [235]:
breakdown_evaluation(eval_df, TYPE_QAS='entailment only')

Jawaban benar (answer exist) entailment: 197
Jawaban benar (answer exist) neutral: 53
Jawaban benar (answer exist) contradiction: 17

Jawaban salah (answer exist) entailment: 180
Jawaban salah (answer exist) neutral: 192
Jawaban salah (answer exist) contradiction: 106

Jawaban prediksi no-answer benar: 12
Jawaban prediksi no-answer salah: 7

Jawaban prediksi benar, tapi tidak masuk prediksi final (menjadi salah) karena label NLI yang tidak sesuai: 70
Jawaban prediksi salah, tapi masuk prediksi final (menjadi benar) karena label NLI yang sesuai: 239

Jawaban prediksi benar (unanswerable), tapi tidak masuk prediksi final (menjadi salah) karena label NLI yang tidak sesuai: 5
Jawaban prediksi salah (unanswerable), tapi masuk prediksi final (menjadi benar) karena label NLI yang sesuai: 0

Total prediksi jawaban: 764



In [227]:
eval_df['Label After Filtering'][2][0]['score']

0.9994680285453796